### 4.4.6. 허깅페이스

In [3]:
using PyCall
pytf = pyimport("transformers")

PyObject <module 'transformers' from '/home/tyfun/.local/lib/python3.10/site-packages/transformers/__init__.py'>

In [4]:
# 파이썬에 transformers 모듈이 설치되어 있지 않은 경우
#@pyimport pip
#pip.main(["install", "transformers"])

BERT base model (uncased)

In [6]:
unmasker = pytf.pipeline("fill-mask", model="bert-base-uncased")
unmasker("Hello I'm a [MASK] model.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


5-element Vector{Dict{Any, Any}}:
 Dict("score" => 0.10731087625026703, "token" => 4827, "token_str" => "fashion", "sequence" => "hello i'm a fashion model.")
 Dict("score" => 0.08774468302726746, "token" => 2535, "token_str" => "role", "sequence" => "hello i'm a role model.")
 Dict("score" => 0.053383879363536835, "token" => 2047, "token_str" => "new", "sequence" => "hello i'm a new model.")
 Dict("score" => 0.046672191470861435, "token" => 3565, "token_str" => "super", "sequence" => "hello i'm a super model.")
 Dict("score" => 0.027095822617411613, "token" => 2986, "token_str" => "fine", "sequence" => "hello i'm a fine model.")

In [10]:
using CSV, DataFrames
imdb = CSV.read("/home/tyfun/Downloads/IMDB Dataset.csv", DataFrame);
text = replace.(imdb.review, "<br />" => " ")[1:5];
tokenizer = pytf.BertTokenizer.from_pretrained("bert-base-uncased");
model = pytf.BertModel.from_pretrained("bert-base-uncased");
encoded_input = tokenizer(text, return_tensors="pt", 
    padding=true, truncation=true);
output = model(
    input_ids = encoded_input.input_ids,
    token_type_ids = encoded_input.token_type_ids,
    attention_mask = encoded_input.attention_mask);
features = output["last_hidden_state"].detach().numpy()[:,1,:]'

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


768×5 adjoint(::Matrix{Float32}) with eltype Float32:
  0.183457   -0.107541     -0.249168     0.0909832   -0.2782
 -0.416676    0.053653     -0.35873     -0.463723    -0.193687
 -0.308941   -0.0827718    -0.246794     0.139111     0.0527724
 -0.212909   -0.46497       0.143973     0.152972    -0.016622
 -0.472088   -0.031428     -0.318534    -0.220803     0.175561
 -0.395376   -0.460735     -0.483444    -0.600356    -0.760108
  0.174218   -0.0703339    -0.205374     0.187732     0.00556535
  0.363432    0.337521      0.511941     0.432029     0.29052
  0.458574    0.382625      0.822231     0.175565     0.429204
 -0.236621   -0.0891502    -0.290467    -0.369064    -0.890835
  ⋮                                                  
 -0.797983   -0.477678     -0.359532    -0.445047    -0.420487
 -0.293951   -0.000521873  -0.0718453    0.00862555   0.190004
 -0.614028   -0.397844     -0.285977    -0.441709    -0.556027
  0.0472205  -0.501228     -0.285543    -0.130639    -0.776218
  0.958017

In [ ]:
dic = FileIO.load("imdb_bert_feat.jld2");
feat_prev = dic["X"][:,1:5]

In [ ]:
using Distances
colwise(cosine_dist, feat_prev, features)

GPT-2

In [11]:
generator = pytf.pipeline("text-generation", model="gpt2")
generator("I like to write julia code", max_length=30, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


5-element Vector{Dict{Any, Any}}:
 Dict("generated_text" => "I like to write julia code. See julia's julia documentation for more information. The following is what it looks like:\n\n\$")
 Dict("generated_text" => "I like to write julia code, but it's easy to forget about jQuery because its built in framework. Here it is, using a javascript,")
 Dict("generated_text" => "I like to write julia code but sometimes you're doing something that is too late or something is broken. So it's sometimes hard for me to")
 Dict("generated_text" => "I like to write julia code and I write it in python, but at the same time I don't want my people to know that julia")
 Dict("generated_text" => "I like to write julia code. I'm using JVM for this because I'm a Java IDE for Java developers. When I'm talking more")

BART

In [12]:
summarizer = pytf.pipeline("summarization", model="facebook/bart-large-cnn")

Downloading: 100%|██████████| 1.58k/1.58k [00:00<00:00, 1.59MB/s]
Downloading: 100%|██████████| 1.63G/1.63G [01:10<00:00, 23.0MB/s]
Downloading: 100%|██████████| 899k/899k [00:00<00:00, 945kB/s] 
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 492kB/s]
Downloading: 100%|██████████| 1.36M/1.36M [00:01<00:00, 1.19MB/s]


PyObject <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x7f7ba873ffd0>

In [13]:
ARTICLE = """ 
At least 10 people were killed and 55 were injured in the southern Ukrainian city of Kherson after Russian shelling hit the area on Saturday, Yaroslav Yanushevych, head of the Kherson region military administration, said in a new update on the death toll.
Yanushevych called on Kherson residents to donate blood to help save the lives of people injured in the shelling. 
He also said the Russians used "Grad multiple-launch rocket systems to hit Kherson city center," and 18 people, among 55, are in critical condition at the moment. 
The head of the regional administration noted there were no children among the victims of Saturday morning's shelling.
At the same time, he reported five people were killed and 17 were injured Friday as a result of Russian shelling.
"Among the injured is a six-year-old girl, doctors fought for her life and managed to save her," Yanushevych said. "But, unfortunately, the six-year-old girl lost her eye and her ear. She has a broken leg. We will evacuate her to Kyiv." 
Yanushevych said the shelling of the city continued all day Saturday, especially the areas along the Dnipro river, and it prevented the rescuers from doing their job of clearing the rubble.
The areas along the Dnipro river, such as Hydropark, Antonivka and Navtohavan have no heating or electricity. Yanushevych said people who live in these areas are under constant heavy shelling and "need to immediately evacuate." 
"It is almost impossible to restore the infrastructure in these areas," Yanushevych said.
""";

In [14]:
summarizer(ARTICLE, max_length=130, min_length=30)

1-element Vector{Dict{Any, Any}}:
 Dict("summary_text" => "At least 10 people were killed and 55 were injured in the southern Ukrainian city of Kherson. 18 people, among 55, are in critical condition at the moment. The head of the regional administration noted no children among the victims.")